In [7]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
data = pd.read_csv("C:/Users/Jintae/Desktop/python_regression/YEAR_DATA.csv", encoding="cp949")
data.head()

,일자,요일,시간대,업종,시도,시군구,읍면동,통화건수
0,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강남구,역삼동,5
1,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강동구,길동,5
2,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강북구,번동,5
3,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강서구,가양동,5
4,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강북구,미아동,5


In [9]:
# 계절 추가
date = list(data.일자)
season = list()

for x in date:
    month = int(x % 10000 / 100)
    if month in [3, 4, 5]:
        season.append('봄')
    elif month in [6, 7 ,8]:
        season.append('여름')
    elif month in [6, 7 ,8]:
        season.append('가을')
    else:
        season.append('겨울')
    
data['계절'] = season
data.head()

,일자,요일,시간대,업종,시도,시군구,읍면동,통화건수,계절
0,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강남구,역삼동,5,겨울
1,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강동구,길동,5,겨울
2,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강북구,번동,5,겨울
3,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강서구,가양동,5,겨울
4,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강북구,미아동,5,겨울


In [11]:
# 공휴일 추가
holiday_list = [20180101, 20180215, 20180216, 20180217, 20180301, 20180505, 20180522, 20180606, 20180815, 20180923, 20180924, 20180925, 20181003, 20181009, 20181225]

date = list(data.일자)
holiday = list()

for x in date:
    if x in holiday_list:
        holiday.append(1)
    else:
        holiday.append(0)
    
data['공휴일'] = holiday
data.head()

,일자,요일,시간대,업종,시도,시군구,읍면동,통화건수,계절,공휴일
0,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강남구,역삼동,5,겨울,1
1,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강동구,길동,5,겨울,1
2,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강북구,번동,5,겨울,1
3,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강서구,가양동,5,겨울,1
4,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강북구,미아동,5,겨울,1


In [12]:
# 일자 -> 월로 바꾸기

date = list(data.일자)
months = list()

for x in date:
    month = int(x % 10000 / 100)
    months.append(month)
    
data['월'] = months
data.head()

,일자,요일,시간대,업종,시도,시군구,읍면동,통화건수,계절,공휴일,월
0,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강남구,역삼동,5,겨울,1,1
1,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강동구,길동,5,겨울,1,1
2,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강북구,번동,5,겨울,1,1
3,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강서구,가양동,5,겨울,1,1
4,20180101,월,0,음식점-족발/보쌈전문,서울특별시,강북구,미아동,5,겨울,1,1


In [13]:
data_simple = data.drop('일자', axis=1)
data_simple = data_simple.drop('시도', axis=1)
data_simple = data_simple.drop('시군구', axis=1)
data_simple = data_simple.drop('읍면동', axis=1)
data_simple.head()

,요일,시간대,업종,통화건수,계절,공휴일,월
0,월,0,음식점-족발/보쌈전문,5,겨울,1,1
1,월,0,음식점-족발/보쌈전문,5,겨울,1,1
2,월,0,음식점-족발/보쌈전문,5,겨울,1,1
3,월,0,음식점-족발/보쌈전문,5,겨울,1,1
4,월,0,음식점-족발/보쌈전문,5,겨울,1,1


In [14]:
data_c = data_simple[data_simple['업종'] == '치킨']
data_c.head()

,요일,시간대,업종,통화건수,계절,공휴일,월
68,월,0,치킨,14,겨울,1,1
69,월,0,치킨,5,겨울,1,1
70,월,0,치킨,5,겨울,1,1
71,월,0,치킨,33,겨울,1,1
72,월,0,치킨,5,겨울,1,1


In [15]:
# 다중공선성 확인을 위한 상관 계수 측정
data_corr = data_c.drop('업종', axis=1)
data_corr = data_corr.drop('통화건수', axis=1)
#data_corr.head()

data_corr = data_corr.assign(요일=data_corr.요일.astype('category').cat.codes)\
.assign(시간대=data_corr.시간대.astype('category').cat.codes)\
.assign(계절=data_corr.계절.astype('category').cat.codes)\
.assign(공휴일=data_corr.공휴일.astype('category').cat.codes)\
.assign(월=data_corr.월.astype('category').cat.codes)

data_corr.corr()

,요일,시간대,계절,공휴일,월
요일,1.000000,-0.023423,-0.012210,0.052041,-0.001574
시간대,-0.023423,1.000000,0.002235,-0.011755,-0.007014
계절,-0.012210,0.002235,1.000000,-0.064264,-0.082935
공휴일,0.052041,-0.011755,-0.064264,1.000000,-0.015796
월,-0.001574,-0.007014,-0.082935,-0.015796,1.000000


In [16]:
# one-hot encoding
data_dummy = pd.get_dummies(data=data_c, columns=['요일'], drop_first=True)
data_dummy = pd.get_dummies(data=data_dummy, columns=['시간대'], drop_first=True)
data_dummy = pd.get_dummies(data=data_dummy, columns=['계절'], drop_first=True)
#data_dummy = pd.get_dummies(data=data_dummy, columns=['공휴일'], drop_first=True)
data_dummy = pd.get_dummies(data=data_dummy, columns=['월'], drop_first=True)

data_dummy = data_dummy.drop('업종', axis=1)

data_dummy.head()

,통화건수,공휴일,요일_목,요일_수,요일_월,요일_일,요일_토,요일_화,시간대_1,시간대_2,...,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12
68,14,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69,5,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,5,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71,33,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72,5,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
features = data_dummy.drop('통화건수', axis=1)

X_data = features.values
y_data = data_c['통화건수'].values
y_data = np.reshape(y_data, (-1, 1))

print(X_data.shape, y_data.shape)

(460175, 43) (460175, 1)


In [39]:
X = tf.placeholder(tf.float32, shape=[None, 43])
y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([43, 1]), name='weight')
b = tf.Variable(tf.random_normal([1], name='bias'))

# Hypothesis
hypothesis = tf.matmul(X, W) + b
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - y))
# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(10001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: X_data, y: y_data})
    if step % 1000 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  466.37164 
Prediction:
 [[ 1.0086403 ]
 [ 1.0086403 ]
 [ 1.0086403 ]
 ...
 [-0.41217002]
 [-0.41217002]
 [-0.41217002]]
1000 Cost:  457.53644 
Prediction:
 [[ 1.2960125 ]
 [ 1.2960125 ]
 [ 1.2960125 ]
 ...
 [-0.10178863]
 [-0.10178863]
 [-0.10178863]]
2000 Cost:  449.18274 
Prediction:
 [[1.5752884 ]
 [1.5752884 ]
 [1.5752884 ]
 ...
 [0.19986248]
 [0.19986248]
 [0.19986248]]
3000 Cost:  441.2841 
Prediction:
 [[1.8467107 ]
 [1.8467107 ]
 [1.8467107 ]
 ...
 [0.49302417]
 [0.49302417]
 [0.49302417]]
4000 Cost:  433.8153 
Prediction:
 [[2.1104968]
 [2.1104968]
 [2.1104968]
 ...
 [0.7779357]
 [0.7779357]
 [0.7779357]]
5000 Cost:  426.75278 
Prediction:
 [[2.3668554]
 [2.3668554]
 [2.3668554]
 ...
 [1.0548282]
 [1.0548282]
 [1.0548282]]
6000 Cost:  420.0744 
Prediction:
 [[2.6159902]
 [2.6159902]
 [2.6159902]
 ...
 [1.3239187]
 [1.3239187]
 [1.3239187]]
7000 Cost:  413.75873 
Prediction:
 [[2.858109]
 [2.858109]
 [2.858109]
 ...
 [1.585437]
 [1.585437]
 [1.585437]]
8000 Cost:  407.

In [41]:
total_error = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y, hy_val)))
R_squared = tf.subtract(1, tf.div(unexplained_error, total_error))
print(R_squared)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


TypeError: Input 'y' of 'Sub' Op has type float32 that does not match type int32 of argument 'x'.